# RNN

text

In [ ]:
# RNN (Recurrent Neural Network)

- http://blog.naver.com/PostView.nhn?blogId=magnking&logNo=221323257045&redirect=Dlog&widgetTypeCall=true&directAccess=false

[simple RNN]
-  cnn(특징추출) - ffnn
- rnn도 마찬가지로 ffnn해줘야함_ 모든 신경망의 기본은 ffnn

- 입력 데이터 받고 => 가중치 => output : 이 통째로 cell이라한다

- 여러개의 cell을 묶어서 output과 연결시킨다 
- 이전데이터 가중치의 영향을 고려한다  > 앞단의 데이터가 모두 끝나야한다는 것 . 
    - 그래서 latent time(지연시간)  발생 
    - 각 cell의 output이 연속족으로 최종 output까지 영향을 미쳐야하는데
    - 중간에 영향력을 잃거나 중복적으로 영향을 미치는 경우가 있다
    
[Long Short Time Memory]: 장기기억 _ https://wikidocs.net/22888
- 그래서 나온게 LSTM : 처음의 영향력을 잃지않기위해 사용
    - STATE를 2개 준다 : CONTROL STATE , HIDDEN STATE
    - HIDDEN STATE 가 OUTPUT: 지속적으로 다음요소에 영향을 미칠것을 결정?
    - 하나는 다음 회로로 나가고 하나는 output으로 나감?
    - 
- gate 3개 : forget input output
    - forget : 앞단에서 흘러오는 ct-1 를 어떻게 제어할것인가를 결정( 망각 or keep)
        - 현재 들어오는 데이터와 이전 state에서 들어오는 ht-1 을 고려해서 
    - input : control선에 어떻게 변화해야 영향력을 유지할 수 있을지 제어 
    - output : hidden
       
- 출력을 위해서 가중치 4개가 필요 (forget input output 과 CELL 안에서 영향을 끼치는한개(TANH)를 다 계산)
    - 4개의 메모리공간 필요
    - 계속 영향을 미치기는 하지만 가중치가 다름    
    - 가중치 사이즈 계산할 떄 4배해줌
    
    
[ GRU ] : LSTM이 너무 복잡해서 나온것

- FORGET과 INPUT 을 하나로 묶음 :회로를 간략화 , 연산이 줄얻을음
- 순서가 있는 데이터를 다를 수 있다 .-> 시계열 DATA : SEQUENTIAL DATA
    - 나는 /학교에/ 간다 
   
 [CAPTIONING]   첫번째 셀(iNput)에 이미지를 넣어주면 이미지를 captioning하는 것
 
- ONE TO ONE :
- ONE TO MANY : captioning 할 때 사용 
- MANY TO MANY :번역할 때 사용 
- MANY TO MANY :동영상 FRAME에 들어있는 OBJECT를 DETECTION 할 때

[rnn 종류]_ 휴대폰 사진찍어놈
- 홋 RNN: Simple RNN 
- Bi directional RNN: 양쪽방향으로 뽑음
- MULTI RNN:  CELL을 여러개 쌓아서 다층으로_ 다층일수록 정확


#input character => input layer => hidden layer => ouput layer => target character


-CELL
MULTI LSTM CELL

- STATIC RNN :셀 수가 고정
- DYNAMIC RNN : 시간적으로 계산하면되어서 (쭉 늘어놀 필요가 없다 ) 하나계산하고 하나계산하고 쮺쮺


In [11]:
import numpy as np
import os
import tensorflow as tf

# 그라프 리셋할때 이거 써주기
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

n_inputs = 3 # 입력 데이터 사이즈
n_neurons = 5  # 셀의 가중치 사이즈

# 리셋 - 초기화(변수 생성)
reset_graph()
   # 아니면 reset_default_graph() 이걸 해주던가


X0 = tf.placeholder(tf.float32, [None, n_inputs])  # 4x3
X1 = tf.placeholder(tf.float32, [None, n_inputs])  # 4x3

# [0,1,2] 사이즈가 하나의 셀로 입력  BasicRNNCell (기본셀)
# FFNN (feed forward neural network)
# 가중치 사이즈 => 특성을 찾아내는 것  num_units = n_neurons
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units = n_neurons)

# static_rnn : rnn network : 4개의 셀이 연결되면서 메모리 확보
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, 
                                                [X0, X1], dtype = tf.float32)  # 2x4x3 : 배치사이즈(순차)x 셀수 x뉴런수

# 출력, 다음셀로 전달되는 값 (두개를 전달)    # states값은 마지막에 전달되어지는 것   (수평을 셀을 연결)
Y0, Y1 = output_seqs

init = tf.global_variables_initializer()

X0_batch = np.array([[0,1,2], [3,4,5],[6,7,8],[9,0,1]])
X1_batch = np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]])


In [16]:
with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0,Y1], feed_dict = {X0:X0_batch, X1:X1_batch})
    
print("처음 데이터 특성 : ", "\n",Y0_val, "차수:",Y0_val.shape)
print()
print("두번째 데이터 특성 :", "\n"  , Y1_val, "차수:",Y1_val.shape)

# 4x3 => 4x5

처음 데이터 특성 :  
 [[ 0.30741334 -0.32884315 -0.6542847  -0.9385059   0.52089024]
 [ 0.99122757 -0.9542541  -0.7518079  -0.9995208   0.9820235 ]
 [ 0.9999268  -0.99783254 -0.8247353  -0.9999963   0.99947774]
 [ 0.996771   -0.68750614  0.8419969   0.9303911   0.8120684 ]] 차수: (4, 5)

두번째 데이터 특성 : 
 [[ 0.99998885 -0.99976057 -0.06679279 -0.9999803   0.99982214]
 [-0.65249425 -0.51520866 -0.37968954 -0.5922594  -0.08968391]
 [ 0.998624   -0.99715203 -0.03308626 -0.9991566   0.9932902 ]
 [ 0.99681675 -0.9598194   0.39660627 -0.8307606   0.79671973]] 차수: (4, 5)


셀이 가지고 있는 가중치 사이즈는?    
3x5

Image

In [26]:
reset_graph()

# 이미지 784 (28x28)
n_steps = 28   # 노드 수 
n_inputs = 28  # 노드당 인풋 사이즈
n_neurons = 150  # 뉴런 출력
n_outputs =10   # 확률 사이즈
learning_rate = 0.001
batch_size = 150

# 3차원으로                     배치, 셀수, 셀당 데이터입력수
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])
# FFNN
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units = n_neurons)  # 셀당 150개

# 셀 28개
# state 는 마지막 셀에 수평으로 전달되는 값
# state 는 마지막 셀의 output과 동일
# 28개의 셀이 있는데 마지막 1개의 output을 사용 => many to one
# 감정분류 <- 분류기
# state 의 차수 : 150x150
# output의 차수 : latent time 지연시간을 통해서 계산된 셀의 모든 값의 결합 출력 150x28x150  (? x 배치 x 뉴런수)

# outputs 은 나와서 다시 들어가는 출력값(저장) // states는 맨 마지막에 결과(출력)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype = tf.float32)


# 150개의 특징중에 10개만 추출
# 입력차수, 출력 차수만 지정하면 자동으로 바이어스를 생성
# 가중치 공간을 확보해서
# 150x150 => 150x10
# dense의 가중치 사이즈 = 150x10
logits = tf.layers.dense(states, n_outputs)

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits) # 분류를 위한 미분이 가능한 식

loss = tf.reduce_mean(xentropy) # 배치사이즈 => 평균을 통해서 loss (지역해 방지)

# Adam = momentum + propgrad (경사하강법)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)

training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y,1)  # 가장 큰값
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()


from tensorflow.examples.tutorials.mnist import input_data

# 원래 이미지 모양
mnist = input_data.read_data_sets("/tmp/data")

X_test = mnist.test.images.reshape((-1,n_steps, n_inputs))
y_test = mnist.test.labels

In [27]:
n_epochs = 100


with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(  # 60000/150  (training data)
            mnist.train.num_examples // batch_size):  
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            
            # 이미지 사이즈로 생성
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict = {X:X_batch, y:y_batch})
        acc_train = accuracy.eval(feed_dict = {X:X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict = {X:X_test, y:y_test})
        print(epoch, "Train accuracy: ", acc_train, "Test accuracy:", acc_test)   
        
# 여기서 150개의 배치(특징으로) 10개의 labels(출력값) 을 추출하는게 목표. 150x10 곱해서 10개 뽑아내기

# rnn은 시간, 순차성을 가진 특징추출
# cnn 은 주변을 고려해서 특징추출

0 Train accuracy:  0.93333334 Test accuracy: 0.9311
1 Train accuracy:  0.96666664 Test accuracy: 0.9522
2 Train accuracy:  0.97333336 Test accuracy: 0.9584
3 Train accuracy:  0.96666664 Test accuracy: 0.9638
4 Train accuracy:  0.97333336 Test accuracy: 0.9662
5 Train accuracy:  0.96666664 Test accuracy: 0.9686
6 Train accuracy:  0.96666664 Test accuracy: 0.9678
7 Train accuracy:  0.98 Test accuracy: 0.9728
8 Train accuracy:  0.93333334 Test accuracy: 0.9651
9 Train accuracy:  0.96666664 Test accuracy: 0.967
10 Train accuracy:  0.9866667 Test accuracy: 0.9754
11 Train accuracy:  0.98 Test accuracy: 0.9668
12 Train accuracy:  0.98 Test accuracy: 0.9711
13 Train accuracy:  0.98 Test accuracy: 0.9766
14 Train accuracy:  0.99333334 Test accuracy: 0.976
15 Train accuracy:  0.98 Test accuracy: 0.9731
16 Train accuracy:  0.9866667 Test accuracy: 0.975
17 Train accuracy:  0.98 Test accuracy: 0.97
18 Train accuracy:  0.98 Test accuracy: 0.9786
19 Train accuracy:  0.96 Test accuracy: 0.9737
20 Tr

# Multi RNN Cell

In [97]:
# BasicRNNCell : 기본데이터가 입력
# multi rnn cell : 수직으로 레이어 구성 ( 수직으로 셀이 구성 )
# dynamic_rnn : 모델


reset_graph()
n_steps = 28
n_inputs = 28
n_outputs = 10
learning_rate = 0.001
batch_size = 150

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

n_neurons = 100
n_layers = 3  # 3개의 멀티 레이어 :3개의 셀을 생성

layers = [tf.contrib.rnn.BasicRNNCell(num_units = n_neurons, activation = tf.nn.relu)
         for layer in range(n_layers)]

# 3개의 셀을 조합해서 MultiRNNCell (수직으로 쌓는다)
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)

dynamic_rnn: 셀로 입력되는 데이터의 개수에 맞추어서 셀을 구성  

예시)   
나는 학교에 간다.  <- (2자  3자  2자) : 사이즈가 다르다.     
so, static_rnn 은 동일한 사이즈로 맞춘다 : 큰것 기준, 작은걸 padding해서      
    입력사이즈 변동 = 가중치 조절 => 나가는 특징은 일치 (neural 수가 같다.)


text를 숫자화   

- embedding   
- one-hot-encoding   
- tfidfcount?   

In [34]:
# dynamic_rnn: 셀로 입력되는 데이터의 개수에 맞추어서 셀을 구성 


# 3층 셀이 28개가 조성 : 고정 사이즈
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype = tf.float32)

# states 가 몇개가 발생하는가?
# 3 x 150 x 100(neuron)  : 특징을 3번 추출(multi-layer하면)
#                          열방향
states_concat = tf.concat(axis = 1, values = states)  # 150 x 300( multi-layer 해서 3층이 쌓여진걸 사이즈 맞추기 )

# 가중치 학습 FFNN 망
logits = tf.layers.dense(states_concat, n_outputs)  # 가중치 :  300 x 10

# 150 X 10   확률값 (: 원핫인코딩 * log( 예측값 ))
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits)

loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)

# MSE(최적값)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y,1)  # tf.nn.in_top_k : 가장 큰값
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()

ValueError: Variable rnn/multi_rnn_cell/cell_0/basic_rnn_cell/kernel/Adam/ already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\ICT01_17\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 2005, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\Users\ICT01_17\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 3616, in create_op
    op_def=op_def)
  File "C:\Users\ICT01_17\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py", line 507, in new_func
    return func(*args, **kwargs)
  File "C:\Users\ICT01_17\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 788, in _apply_op_helper
    op_def=op_def)
  File "C:\Users\ICT01_17\Anaconda3\lib\site-packages\tensorflow\python\ops\gen_state_ops.py", line 1608, in variable_v2
    shared_name=shared_name, name=name)


In [33]:
n_epochs = 10

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(  # 60000/150  (training data)
            mnist.train.num_examples // batch_size):  
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            
            # 이미지 사이즈로 생성
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict = {X:X_batch, y:y_batch})
        acc_train = accuracy.eval(feed_dict = {X:X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict = {X:X_test, y:y_test})
        print(epoch, "Train accuracy: ", acc_train, "Test accuracy:", acc_test)  

0 Train accuracy:  0.94666666 Test accuracy: 0.9365
1 Train accuracy:  0.96 Test accuracy: 0.9486
2 Train accuracy:  0.96666664 Test accuracy: 0.9601
3 Train accuracy:  0.98 Test accuracy: 0.9725
4 Train accuracy:  0.97333336 Test accuracy: 0.9614
5 Train accuracy:  0.99333334 Test accuracy: 0.974
6 Train accuracy:  0.97333336 Test accuracy: 0.9776
7 Train accuracy:  0.9866667 Test accuracy: 0.977
8 Train accuracy:  0.99333334 Test accuracy: 0.9803
9 Train accuracy:  0.9866667 Test accuracy: 0.9729


# 다른 예시
mnist를 RNN 으로 분류해보기     
http://blog.naver.com/PostView.nhn?blogId=magnking&logNo=221323257045&redirect=Dlog&widgetTypeCall=true&directAccess=false

(아래 코드, 위 사이트 참고)    

In [101]:
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot = True)
learning_rate = 0.001 # 경사하강법을 사용하겠다
total_epoch = 10
batch_size = 128  # 6만장 중에서 임의의 128장을 뽑는다

n_input = 28
n_step = 28
n_hidden = 128  # hidden = neuron
n_class = 10

X = tf.placeholder(tf.float32, [None, n_step, n_input])  # 128x28x28
Y = tf.placeholder(tf.float32, [None, n_class])         # 128x10
W = tf.Variable(tf.random_normal([n_hidden, n_class]))  # 128x10
b = tf.Variable(tf.random_normal([n_class]))   # 10

# 28x128 사이즈의 가중치 통과하면 -> # neural: 128특징
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)  

# cell 수 : 28개
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype = tf.float32)

# FFNN 망
# output 사이즈 : 128 x 28 x 128   (각셀마다 뉴런이 128개/// 즉, batch(128) x step(28) x hidden(128))
# state : 128 x 128
outputs = tf.transpose(outputs, [1,0,2]) # output에서 0과1이 바뀌어서: 28x128x128

# cell로 판단( 셀은 28개 )
outputs = outputs[-1]  # 맨 마지막 셀: 128 x 128  

model = tf.matmul(outputs, W) + b   # 128x128  , 128x10  => 128x10

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)


Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [102]:
sess = tf.Session()  
sess.run(tf.global_variables_initializer())  # 변수 초기화
total_batch = int(mnist.train.num_examples//batch_size)  # 60000/128의 몫

for epoch in range(total_epoch): # 10번
    total_cost = 0
    
    for i in range(total_batch):  # 한번 돌릴때 전체 사이즈 돌린다
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        # 원래 사이즈로 되돌리기
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
        
      # _ : python이 계산한 마지막 결과가 저장되는 것                     
        _, cost_val = sess.run([optimizer, cost],   # 경사하강법(optimizer)
                              feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += cost_val
        
    print('Epoch:', '%04d' %(epoch +1), 
          'Avg.cost = ', '{:.3f}'.format(total_cost/total_batch))
    print('최적화 완료')      

Epoch: 0001 Avg.cost =  0.636
최적화 완료
Epoch: 0002 Avg.cost =  0.265
최적화 완료
Epoch: 0003 Avg.cost =  0.197
최적화 완료
Epoch: 0004 Avg.cost =  0.159
최적화 완료
Epoch: 0005 Avg.cost =  0.152
최적화 완료
Epoch: 0006 Avg.cost =  0.130
최적화 완료
Epoch: 0007 Avg.cost =  0.122
최적화 완료
Epoch: 0008 Avg.cost =  0.113
최적화 완료
Epoch: 0009 Avg.cost =  0.106
최적화 완료
Epoch: 0010 Avg.cost =  0.104
최적화 완료


문제
1. 테스트 데이터를 이용하여 테스트 하는 회로를 구성하시오.
2. 3개의 셀을 갖는 multi-layer cell로 수정하시오.

In [46]:
# 2번
n_layers = 3

# 3개의 셀로 멀티레이어를 생성해도 output은 하나의 셀일 경우와 동일
# state에서는 3개의 레이어마다 생성
layers = [tf.contrib.rnn.BasicRNNCell(num_units = n_hidden, activation = tf.nn.relu)
         for layer in range(n_layers)]

multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)

outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype = tf.float32)

outputs = tf.transpose(outputs, [1,0,2])   # 3개로 나오니까 변경하기

# 1 번?
# 세션이 아직 열려있어서 다시 열지 않아도 된다
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_corect, tf.float32))
test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels
print("정확도 : ", sess.run(accuracy, feed_dict = {X:test_xs, Y:test_ys}))

ValueError: in converted code:
    relative to C:\Users\ICT01_17\Anaconda3\lib\site-packages\tensorflow\python:

    ops\rnn_cell_impl.py:1719 call
        cur_inp, new_state = cell(cur_inp, cur_state)
    ops\rnn_cell_impl.py:385 __call__
        self, inputs, state, scope=scope, *args, **kwargs)
    layers\base.py:537 __call__
        outputs = super(Layer, self).__call__(inputs, *args, **kwargs)
    keras\engine\base_layer.py:591 __call__
        self._maybe_build(inputs)
    keras\engine\base_layer.py:1881 _maybe_build
        self.build(input_shapes)
    keras\utils\tf_utils.py:295 wrapper
        output_shape = fn(instance, input_shape)
    ops\rnn_cell_impl.py:455 build
        shape=[input_depth + self._num_units, self._num_units])
    keras\engine\base_layer.py:1484 add_variable
        return self.add_weight(*args, **kwargs)
    layers\base.py:450 add_weight
        **kwargs)
    keras\engine\base_layer.py:384 add_weight
        aggregation=aggregation)
    training\tracking\

# LSTM

In [95]:
import numpy as np
values231 = np.array([
    [[1],[2],[3]],
    [[2],[3],[4]]
])
print("입력 데이터의 shape", values231.shape) # 2x3x1

import tensorflow as tf
tf.reset_default_graph()
tf_values231 =tf.constant(values231, dtype = tf.float32)

# 1x100 의 가중치가 4개가 생성되어야 한다
lstm_cell = tf.contrib.rnn.LSTMCell(num_units = 100) # 뉴론

# rnn 모델은 동일 : 망과 망을 연결할때
# 단, state의 값이 2개가 나온다. 
# latent time 지연시간
outputs, state = tf.nn.dynamic_rnn(cell = lstm_cell, dtype = tf.float32, inputs = tf_values231)

# 망과 망을 연결할때 (번역 망 : 한글, 영어) 필요
print(state.c.shape) # 2x100
print(state.h.shape) # 2x100

print("rnn이 출력하는 outputs의 의미", outputs.shape)  # 2x3x100


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_run, state_run = sess.run([outputs, state])

입력 데이터의 shape (2, 3, 1)
(2, 150)
(2, 150)
rnn이 출력하는 outputs의 의미 (2, 3, 150)


In [96]:
tf.reset_default_graph()
values = tf.constant(np.array([
    [[1],[2],[3]],
    [[2],[3],[4]]
]), dtype = tf.float32)
lstm_cell_fw = tf.contrib.rnn.LSTMCell(100)
lstm_cell_bw = tf.contrib.rnn.LSTMCell(100)
# Multi Cell : 셀에서
# 양방향 LSTM
(output_fw, output_bw), (output_state_fw, output_state_bw) = tf.nn.bidirectional_dynamic_rnn(
    cell_fw=lstm_cell_fw,
    cell_bw=lstm_cell_bw,
    inputs=values,
    dtype=tf.float32)
print(output_fw.shape)
print(output_bw.shape)

(2, 3, 100)
(2, 3, 100)


# 단어를 수치화
- embedding
- pprint :
문자열에 포함된 format 함수를 사용하면 일반적으로 가장 보기 좋은 출력 형태를 생성할 수 있다.
그런데, format 함수는 전달해야 할 것이 많아서 번거로운 느낌이 들 때가 많다.
이럴 때 조금 부족하기는 하지만, pprint 모듈을 사용하면 상당 부분 절충하는 것이 가능하다.


In [55]:
# embedding

from tensorflow.contrib import rnn
import pprint
from tensorflow.python.framework import ops

ops.reset_default_graph()
pp = pprint.PrettyPrinter(indent = 4)
sess = tf.InteractiveSession()

# 단어를 원핫인코딩: 수치화
h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]

# 변수 공유를 위해 지정
with tf.variable_scope('five_sequences') as scope:
    hidden_size = 2
    # 4 -> 2개의 특징으로 vector화 된다
    
    cell = tf.contrib.rnn.BasicRNNCell(num_units = hidden_size)
    x_data = np.array([[h,e,l,l,o]], dtype = np.float32)
    
    print(x_data.shape)       # (1, 5, 4): 데이터 한개 => 3차원으로
  #      [[[1., 0., 0., 0.],
  #       [0., 1., 0., 0.],
  #       [0., 0., 1., 0.],
  #       [0., 0., 1., 0.],
  #      [0., 0., 0., 1.]]]    3차원
        
    pp.pprint(x_data)
    # cell의 개수 : 5
    # 셀당 입력데이터는 4
    # 셀당 가중치는 4 x 2
    
    outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype = tf.float32)
        
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
    
    print(outputs.shape)  # 1x5x2
    print(states.shape)   # 1x2  (데이터 한개)
    pp.pprint(states.eval())

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[-0.25850698, -0.54317063],
        [ 0.22713791,  0.3987561 ],
        [ 0.6081036 , -0.5190886 ],
        [ 0.6853291 , -0.10337783],
        [ 0.7947474 , -0.37653482]]], dtype=float32)

(1, 5, 2)
(1, 2)
array([[ 0.7947474 , -0.37653482]], dtype=float32)


C:\Users\ICT01_17\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


# 문자에 번호를 키로주기

In [80]:
ops.reset_default_graph()

char_arr = ['a','b','c','d','e','f','g',
           'h','i','j','k','l','m','n',
           'o','p','q','r','s','t','u',
            'v','w','x','y','z']
# num_index [0,1,2,3,..................,25]

num_dic = {n:i for i, n in enumerate(char_arr)}
# 키를 만들어줌
# a: 0 , b :1번. (번호를 키로)

dic_len = len(num_dic) # 26개
# rule base => 자동 툴 => 가중치만 이용
seq_data = ['word','wood','deep','dive','cold','cool','load','love','kiss','kind']


# 10x3 (3: word에서 wor)
#

In [81]:
# 26x26 행렬 (이렇게)
# 100000000000000000000000000000000...
# 010000000000000000000000000000000...
# 001000000000000000000000000000000...


# w
# 000000000000000000000000000000010...  (22번)
# 000000000000010000000000000000000...  (14번)
# 00000000000000010000000000000000...   (17번)


def make_batch(seq_data):
    input_batch = []
    target_batch = []
    
    for seq in seq_data:  # seq에서 'word' 가져오고 마지막은 제외[:-1]
        input = [num_dic[n] for n in seq[:-1]]  # w: 22번, o:14번, r:17번, (마지막 d는 제외)
        target = num_dic[seq[-1]]   # d :3번
        input_batch.append(np.eye(dic_len)[input])  # 원핫인코딩
        target_batch.append(target)
    return input_batch, target_batch

make_batch(seq_data[0])

([array([], shape=(0, 26), dtype=float64),
  array([], shape=(0, 26), dtype=float64),
  array([], shape=(0, 26), dtype=float64),
  array([], shape=(0, 26), dtype=float64)],
 [22, 14, 17, 3])

In [82]:
# 학습시키기

learning_rate = 0.01
n_hidden = 128
total_epoch = 30
n_step = 3
n_input = n_class = dic_len  # 26개

X = tf.placeholder(tf.float32, [None, n_step, n_input])  # 10x3x26
Y = tf.placeholder(tf.int32, [None]) # 10x26

# FFNN
W = tf.Variable(tf.random_normal([n_hidden, n_class]))  # 128 x 26
b = tf.Variable(tf.random_normal([n_class]))

In [83]:
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden) # 특정수
# 과적합 방지하기 위해 dropout
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob = 0.5,seed=10)

# 2레이어로 구성된 멀티 셀
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1,cell2])
outputs, states = tf.nn.dynamic_rnn(multi_cell,X,dtype = tf.float32)

# 10x3x128
# state : 10 x 128

In [84]:
outputs = tf.transpose(outputs,[1,0,2])  # 3x10x128 맨뒤의 셀
outputs = outputs[-1]  # 10 x128
model = tf.matmul(outputs,W) + b  # 비선형 회귀방식

# 분류
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model,labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [87]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, target_batch = make_batch(seq_data) # 단어데이터


for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],feed_dict = {X:input_batch, Y:target_batch})
       
    print('Epoch:', '%04d' %(epoch +1), 
          'cost = ', '{:.6f}'.format(loss))

Epoch: 0001 cost =  3.864312
Epoch: 0002 cost =  2.913284
Epoch: 0003 cost =  1.485137
Epoch: 0004 cost =  1.805050
Epoch: 0005 cost =  0.863734
Epoch: 0006 cost =  0.965523
Epoch: 0007 cost =  1.035705
Epoch: 0008 cost =  0.581941
Epoch: 0009 cost =  0.405560
Epoch: 0010 cost =  0.634337
Epoch: 0011 cost =  0.320449
Epoch: 0012 cost =  0.332297
Epoch: 0013 cost =  0.503054
Epoch: 0014 cost =  0.443100
Epoch: 0015 cost =  0.157287
Epoch: 0016 cost =  0.150054
Epoch: 0017 cost =  0.188537
Epoch: 0018 cost =  0.050019
Epoch: 0019 cost =  0.105574
Epoch: 0020 cost =  0.153275
Epoch: 0021 cost =  0.067556
Epoch: 0022 cost =  0.046727
Epoch: 0023 cost =  0.162803
Epoch: 0024 cost =  0.102455
Epoch: 0025 cost =  0.050631
Epoch: 0026 cost =  0.048829
Epoch: 0027 cost =  0.027342
Epoch: 0028 cost =  0.029229
Epoch: 0029 cost =  0.007853
Epoch: 0030 cost =  0.010423


In [89]:
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, targe_batch = make_batch(seq_data)
predict, accuracy_val = sess.run([prediction, accuracy], feed_dict = {X:input_batch, Y:target_batch})

문제    
- predict 한 결과를 출력하고 accuracy를 출력하시오.   
- 입력 단어와 예측결과를 나란히 출력해 보시오.

In [94]:
# 문제
print("예측된 결과", predict)
print("Accuracy", accuracy_val)

predict_words = []
# 단어가 순서대로 입력, 순서대로 예측
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]] # 인덱스가 단어 매핑
    predict_words.append(val[:3]+ last_char)
print("입력값:",[w[:3] + ' ' for w in seq_data])
print("예측값:", predict_words)
print("정확도:", accuracy_val)

예측된 결과 [ 3  3 15  4  3 11  3  4 18  3]
Accuracy 1.0
입력값: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도: 1.0
